In [26]:
import pandas as pd
import numpy as np
df = pd.read_excel('fund_data.xlsx')

In [27]:
# 연환산 수익률 계산 함수
def annualized_return(cumulative_return, periods, period_type='year'):
    """
    누적 수익률을 연환산 수익률로 변환합니다.
    
    :param cumulative_return: 누적 수익률 (%)
    :param periods: 투자 기간 (년 단위)
    :param period_type: 'month' 또는 'year'
    :return: 연환산 수익률 (%)
    """
    if period_type == 'month':
        n = periods / 12
    elif period_type == 'year':
        n = periods
    else:
        raise ValueError("period_type은 'month' 또는 'year'이어야 합니다.")
    
    if n <= 0:
        return 0
    return ((1 + cumulative_return / 100) ** (1 / n) - 1) * 100

# 펀드종류 점수 계산 함수
def fund_type_score(fund_type, preferred_types):
    return 100 if fund_type.lower() in [ptype.lower() for ptype in preferred_types] else 0

# 수익률 점수 계산 함수
def calculate_return_score(row, investment_period):
    return get_selected_return(row, investment_period)

def get_selected_return(row, investment_period):
    """
    투자 기간에 따라 적절한 수익률을 선택하고, 결측치가 있을 경우 대체 수익률을 사용합니다.
    
    :param row: 데이터프레임의 행
    :param investment_period: 투자 기간 (년 단위)
    :return: 선택된 수익률 (%)
    """
    if investment_period <= 1:
        # 1년 이하 투자자는 6개월 수익률 사용
        if not pd.isna(row['6개월수익률']):
            return row['6개월수익률']
        elif not pd.isna(row['1년수익률']):
            return row['1년수익률']
    elif 1 < investment_period <= 3:
        # 1년 초과 ~ 3년 이하 투자자는 1년 수익률 사용
        if not pd.isna(row['1년수익률']):
            return row['1년수익률']
        elif not pd.isna(row['6개월수익률']):
            return row['6개월수익률']
    else:
        # 3년 초과 투자자는 3년 수익률 사용
        if not pd.isna(row['3년수익률']):
            return row['3년수익률']
        elif not pd.isna(row['1년수익률']):
            return row['1년수익률']


In [31]:
# 고객 프로파일 정의
customer_profiles = {
    1: {
        'risk_tolerance': 'low',
        'investment_period': 1,
        'expected_return': 'capital_preservation',
        'weights': {'risk_score': 0.5, 'return_score': 0.2, 'fund_type_score': 0.2, 'fee_score': 0.1},
        'preferred_fund_types': ['기타','ETF','채권형', 'MMF']
    },
    2: {
        'risk_tolerance': 'high',
        'investment_period': 1,
        'expected_return': 'high_return',
        'weights': {'risk_score': 0.2, 'return_score': 0.6, 'fund_type_score': 0.15, 'fee_score': 0.05},
        'preferred_fund_types': ['기타','ETF','주식형']
    },
    3: {
        'risk_tolerance': 'medium',
        'investment_period': 3,
        'expected_return': 'market_return',
        'weights': {'risk_score': 0.3, 'return_score': 0.4, 'fund_type_score': 0.2, 'fee_score': 0.1},
        'preferred_fund_types': ['기타','ETF','혼합형']
    },
    4: {
        'risk_tolerance': 'high',
        'investment_period': 5,
        'expected_return': 'above_market_return',
        'weights': {'risk_score': 0.2, 'return_score': 0.5, 'fund_type_score': 0.2, 'fee_score': 0.1},
        'preferred_fund_types': ['기타','ETF','주식형', '혼합형']
    },
    5: {
        'risk_tolerance': 'low',
        'investment_period': 5,
        'expected_return': 'stable_income',
        'weights': {'risk_score': 0.5, 'return_score': 0.3, 'fund_type_score': 0.15, 'fee_score': 0.05},
        'preferred_fund_types': ['기타','ETF','채권형', 'MMF']
    }
}


In [34]:
# 위험 점수 계산 (1등급 -> 0점, 6등급 -> 100점)
df['risk_score'] = (df['펀드위험'] - 1) / (6 - 1) * 100

# 수수료 점수 계산 (수수료가 낮을수록 높은 점수)
df['fee_score'] = (df['수수료'].max() - df['수수료']) / (df['수수료'].max() - df['수수료'].min()) * 100

# 펀드 추천 함수
def recommend_funds(df, profile_id):
    profile = customer_profiles[profile_id]
    temp_df = df.copy()
    
    # 투자 기간에 따른 수익률 선택 및 점수 계산
    investment_period = profile['investment_period']
    temp_df['selected_return'] = temp_df.apply(lambda row: calculate_return_score(row, investment_period), axis=1)
    
    # 선택된 수익률을 기반으로 점수화 (0~100)
    min_return = temp_df['selected_return'].min()
    max_return = temp_df['selected_return'].max()
    if max_return - min_return != 0:
        temp_df['return_score'] = (temp_df['selected_return'] - min_return) / (max_return - min_return) * 100
    else:
        temp_df['return_score'] = 50  # 모든 수익률이 동일한 경우 중간 점수 부여
    
    # 펀드종류 점수 계산
    temp_df['fund_type_score'] = temp_df['펀드종류'].apply(lambda x: fund_type_score(str(x), profile['preferred_fund_types']))
    
    # 최종 점수 계산
    temp_df['total_score'] = (
        temp_df['risk_score'] * profile['weights']['risk_score'] +
        temp_df['return_score'] * profile['weights']['return_score'] +
        temp_df['fund_type_score'] * profile['weights']['fund_type_score'] +
        temp_df['fee_score'] * profile['weights']['fee_score']
    )
    
    # 점수 순으로 정렬하여 상위 펀드 반환
    recommended_funds = temp_df.sort_values(by='total_score', ascending=False)
    
    # 필요한 열만 선택
    recommended_funds = recommended_funds[['펀드종류', '펀드명', 'total_score', '1년수익률']]
    
    return recommended_funds

In [35]:
# 펀드 추천 예시
for profile_id in customer_profiles.keys():
    recommended_funds = recommend_funds(df, profile_id)
    print(f"프로파일 {profile_id} 추천 펀드:")
    display_df = recommended_funds.head(10).reset_index(drop=True)
    print(display_df)
    print("\n")

프로파일 1 추천 펀드:
  펀드종류                                         펀드명  total_score   1년수익률
0  ETF                  KBRISE금융채액티브증권상장지수투자신탁(채권)    85.829214  0.0690
1  ETF  미래에셋TIGERCD금리투자KIS특별자산상장지수투자신탁(CD-파생형)(합성)    85.708327  0.0364
2  ETF                   키움KOSEF단기자금증권상장지수투자신탁[채권]    85.651354  0.0390
3  ETF                한화PLUSKOFR금리특별자산상장지수투자신탁(금리)    85.631784  0.0350
4  채권형                     브이아이뉴굿초이스우량단기증권투자신탁[채권]    85.628627  0.0419
5  ETF                KBRISE단기국공채액티브증권상장지수투자신탁(채권)    85.613950  0.0406
6  ETF                   한국투자ACE단기자금증권상장지수투자신탁(채권)    85.591934  0.0373
7  ETF                 한화PLUS단기채권액티브증권상장지수투자신탁(채권)    85.584596  0.0429
8  ETF              미래에셋TIGER단기채권액티브증권상장지수투자신탁(채권)    85.579703  0.0377
9  ETF                 신한SOLKIS단기통안채증권상장지수투자신탁[채권]    85.557688  0.0348


프로파일 2 추천 펀드:
  펀드종류                                            펀드명  total_score   1년수익률
0   기타              미래에셋차이나H레버리지2.0증권자투자신탁(주식-파생재간접형)    77.214533  0.3456
1  ETF  미래에셋TIGER차이나항셍테크레버리지